<a href="https://colab.research.google.com/github/stillrahim/jupyter-exploration/blob/main/vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this first in Colab / local notebook
!pip install vaderSentiment textblob scikit-learn librosa soundfile transformers torchaudio matplotlib seaborn pandas nltk

# NLTK download for some tokenizers/stopwords if needed
import nltk
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:
import pandas as pd

# Example small dataset for demonstration (replace with class dataset)
data = [
  {"text":"I love this product, it's fantastic!", "label":"positive"},
  {"text":"This was the worst service ever, I'm disappointed.", "label":"negative"},
  {"text":"I suppose it's okay, nothing special.", "label":"neutral"}
]
df = pd.DataFrame(data)
df


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

analyzer = SentimentIntensityAnalyzer()

def vader_label(text):
    s = analyzer.polarity_scores(text)['compound']
    if s >= 0.05: return 'positive'
    if s <= -0.05: return 'negative'
    return 'neutral'

def textblob_label(text):
    b = TextBlob(text).sentiment.polarity
    if b > 0.05: return 'positive'
    if b < -0.05: return 'negative'
    return 'neutral'

df['vader'] = df['text'].apply(vader_label)
df['textblob'] = df['text'].apply(textblob_label)
df


In [ ]:
from sklearn.metrics import classification_report
print("VADER vs truth")
print(classification_report(df['label'], df['vader']))
print("TextBlob vs truth")
print(classification_report(df['label'], df['textblob']))


In [ ]:
# If you have a CSV with columns text,label, load it:
# df = pd.read_csv('/path/to/emotion_dataset.csv')

# For demo, reuse df but expand it in practice
df_ml = df.copy()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(df_ml['text'], df_ml['label'], test_size=0.3, random_state=42)

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('clf', LinearSVC())
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
# Optionally: use sentence-transformers for embeddings (install first)
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np
emb_model = SentenceTransformer('all-MiniLM-L6-v2')  # small, fast

X_emb = emb_model.encode(df_ml['text'].tolist())
# split and train a simple classifier
from sklearn.linear_model import LogisticRegression
X_tr, X_te, y_tr, y_te = train_test_split(X_emb, df_ml['label'], test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000).fit(X_tr, y_tr)
print(classification_report(y_te, clf.predict(X_te)))


In [ ]:
import librosa
import numpy as np

# Example: load one audio file (replace with actual path)
audio_path = '/content/sample.wav'  # upload a sample or provide link
y, sr = librosa.load(audio_path, sr=16000)

# Extract features: MFCC, chroma, zcr, spectral centroid
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
mfcc_mean = mfcc.mean(axis=1)
zcr = librosa.feature.zero_crossing_rate(y).mean()
centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
features = np.hstack([mfcc_mean, zcr, centroid])
features.shape


In [ ]:
# Suppose you prepared features X_audio (n_samples x n_features) and labels y_audio
# For demo:
X_audio = np.array([features])    # replace with full dataset features
y_audio = np.array(['neutral'])   # replace with real labels

# Train/test split and a small classifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Here you'd normally have many samples; this is templated code
# X_tr, X_te, y_tr, y_te = train_test_split(X_audio, y_audio, test_size=0.3)
# clf_audio = RandomForestClassifier().fit(X_tr, y_tr)
# print(classification_report(y_te, clf_audio.predict(X_te)))


In [ ]:
# text embedding (sentence-transformers)
text_samples = ["I am so happy today!"]  # replace with dataset of transcripts
text_emb = emb_model.encode(text_samples)  # shape (n, d_text)

# audio features (use same pipeline from Part 3) -> features shape (n, d_audio)
audio_feats = np.array([features]) # placeholder

# concatenate
X_multi = np.hstack([text_emb, audio_feats])

# train a classifier
clf_multi = LogisticRegression(max_iter=1000)
# clf_multi.fit(X_train_multi, y_train_multi)
# evaluate...


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization 1: Confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# sns.heatmap(cm, annot=True, fmt='d')
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted")
# plt.ylabel("True")
# plt.show()

# Visualization 2: Emotion distribution pie / bar
# counts = df['predicted_emotion'].value_counts()
# counts.plot(kind='bar')
# plt.title("Predicted Emotion Distribution")
# plt.xlabel("Emotion")
# plt.ylabel("Count")
# plt.show()


In [ ]:
# Quick demo: VADER + librosa feature + simple rule: if vader negative and energy high -> angry
text = "I'm really angry about this order."
v_label = vader_label(text)

# audio: compute energy
energy = np.sum(y**2) / len(y)
print("VADER:", v_label, "Energy:", energy)
if v_label == 'negative' and energy > 0.01:
    print("Rule-based multimodal signal: possible anger")
